In [1]:
from import_src import import_src
import_src()

Append /home/ubuntu/khiempm/MLOpsVN/src to sys.path


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import json
from plotly import express as px
import mlflow
import seaborn as sns

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
pd.set_option('display.max_columns', 500)

In [3]:
link_data_test_raw = '../../data/captured_data/phase-2/prob-2/raw/'
train_raw_data = '../../data/raw_data/phase-2/prob-2/raw_train.parquet'

In [4]:
raw_test = pd.DataFrame()
count_ = 0
for file_path in os.listdir(link_data_test_raw):
    if 'parquet' not in file_path or "123.parquet" in file_path:
        continue
    captured_data = pd.read_parquet(f'{link_data_test_raw}{file_path}')
    captured_data['parquet_idx'] = count_
    raw_test = pd.concat([raw_test, captured_data])
    count_ += 1

In [5]:
raw_test

,feature14,feature1,feature9,feature35,feature33,feature29,feature8,feature16,feature28,feature21,...,feature31,feature37,feature6,feature19,feature11,feature27,feature12,feature5,feature20,parquet_idx
0,0.0,0.000009,254.0,24.0,15.0,0.0,0.0,0.000000,0.0,0.000000e+00,...,2.0,0.0,0.0,0.0,5.066666e+07,0.0,0.000000e+00,2.0,0.000000e+00,0
1,0.0,0.000004,254.0,7.0,2.0,0.0,0.0,0.000000,0.0,0.000000e+00,...,2.0,0.0,0.0,0.0,2.000000e+08,0.0,0.000000e+00,2.0,0.000000e+00,0
2,0.0,0.000009,254.0,14.0,14.0,0.0,0.0,0.000000,0.0,0.000000e+00,...,2.0,0.0,0.0,0.0,5.066666e+07,0.0,0.000000e+00,2.0,0.000000e+00,0
3,1.0,0.454651,254.0,4.0,3.0,0.0,268.0,64.424398,0.0,1.130290e+08,...,1.0,0.0,6.0,255.0,8.463635e+03,45.0,3.941485e+03,10.0,1.731973e+09,0
4,0.0,0.001038,31.0,1.0,1.0,0.0,162.0,0.007000,0.0,0.000000e+00,...,0.0,0.0,2.0,0.0,5.009634e+05,81.0,6.242774e+05,2.0,0.000000e+00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.000003,254.0,21.0,20.0,0.0,0.0,0.000000,0.0,0.000000e+00,...,2.0,0.0,0.0,0.0,1.520000e+08,0.0,0.000000e+00,2.0,0.000000e+00,100
996,0.0,0.000940,31.0,2.0,1.0,0.0,178.0,0.002000,0.0,0.000000e+00,...,0.0,0.0,2.0,0.0,6.212766e+05,89.0,7.574468e+05,2.0,0.000000e+00,100
997,0.0,0.001588,31.0,3.0,1.0,0.0,304.0,0.208667,0.0,0.000000e+00,...,0.0,0.0,4.0,0.0,1.964736e+06,76.0,1.148615e+06,4.0,0.000000e+00,100
998,1.0,0.559659,254.0,4.0,1.0,0.0,268.0,98.494203,0.0,2.115931e+09,...,1.0,0.0,6.0,255.0,8.004875e+03,45.0,3.201950e+03,10.0,1.199180e+09,100


In [91]:
raw_train = pd.read_parquet(train_raw_data)
order_cols = raw_train.columns.tolist()
order_cols.remove('label')
raw_test = raw_test[order_cols]

In [7]:
corr_ = raw_train.drop(columns=['feature2', 'feature3', 'feature4', 'label']).corr()

In [13]:
cols = raw_train.drop(columns=['feature2', 'feature3', 'feature4', 'label']).columns.tolist()

In [45]:
drop_corr = ['feature5', 'feature13', 'feature6', 'feature14', 'feature22', 'feature30', 'feature35', 'feature32', 'feature36', 'feature37']

In [46]:
cate_cols = ['feature9', 'feature10', 'feature31']

In [47]:
raw_train['feature31']

0        0.0
1        0.0
2        2.0
3        1.0
4        0.0
        ... 
61836    0.0
61837    1.0
61838    2.0
61839    2.0
61840    2.0
Name: feature31, Length: 61841, dtype: float64

In [32]:
str(raw_train[each].dtype)

'float64'

In [ ]:
drop_cols = ['feature41', 'feature36', 'feature22', 'feature37']

In [95]:
def data_processing_phase2_prob2(data, phase="train"):
    drop_corr = ['feature5', 'feature13', 'feature6', 'feature14', 'feature22', 'feature30', 'feature35', 'feature32', 'feature36', 'feature37']
    convert_cate_cols = ['feature9', 'feature10', 'feature31']
    data = data.drop(columns = drop_corr)
    for col in convert_cate_cols:
        data[col] = data[col].astype(int).astype(str)
    cate_cols = ['feature2', 'feature3', 'feature4']
    for col in cate_cols:
        dict_count = data[col].value_counts().to_dict()
        list_below = [key for key, value in dict_count.items() if value / data.shape[0] < 0.0005]
        if phase == 'test':
            list_below.append('null')
        data[col] = data[col].replace({value: 'other' for value in list_below})
    return data

In [106]:
[each for each in raw_test_['feature4'].unique() if each not in each for each in raw_train_['feature4'].unique()]

[]

In [107]:
def get_num(n):
    return n

dict_ = {}
dict_[1] = get_num

In [109]:
dict_[1](10)

10

In [111]:
import pickle
with open("../../data/train_data/phase-2/prob-2/category_index.pickle", "rb") as file:
    category_features = pickle.load(file)

In [112]:
category_features 

{'feature2': Index(['arp', 'ospf', 'other', 'sctp', 'tcp', 'udp', 'unas'], dtype='object'),
 'feature3': Index(['-', 'dhcp', 'dns', 'ftp', 'ftp-data', 'http', 'other', 'pop3', 'smtp',
        'ssh', 'ssl'],
       dtype='object'),
 'feature4': Index(['CON', 'FIN', 'INT', 'REQ', 'other'], dtype='object')}